## To predict the masks of image, generate binary masks, prediction images with bounding boxes and prediction labels for whole test_dataset

##### The below codes are run only after creating the Individual LUT_GT_Test GeoJSONs using the GT-YOLOLabel_to_GEOJSON.ipynb code.

In [137]:
import os
os.chdir('/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/Rural_WS/yolo8_segmentation_ws')
model_direc =  os.getcwd()
#print(model_direc)

In [138]:
model_lut = 'Rural_WS'
prediction_lut = 'All_Land_Use_Zones_WS'
post_processing_direc = 'Rural-M_2_ALUT_Test'

In [139]:
test_img_direc = os.path.join('/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province', prediction_lut, 'yolo8_segmentation_ws')
#print(test_img_direc)

prediction_dir = os.path.join("/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province",model_lut, "yolo8_segmentation_ws/post_processing", post_processing_direc)
#print(prediction_dir)

In [ ]:
from PIL import Image
from ultralytics import YOLO
from tqdm import tqdm
import cv2
import numpy as np

custom_model_direc = os.path.join(model_direc, 'runs/segment/train/weights/best.pt')
# Load a model
model = YOLO(custom_model_direc) 

#img_dir = os.path.join(home_direc, 'sample')
img_dir = os.path.join(test_img_direc, 'test/images')
img_list = os.listdir(img_dir)

pred_img_save_dir = os.path.join(prediction_dir, 'predict_img') # image save folder
os.makedirs(pred_img_save_dir, exist_ok=True)

pred_mask_save_dir = os.path.join(prediction_dir, 'predict_mask') # mask save folder
os.makedirs(pred_mask_save_dir, exist_ok=True)

# Predict for a single image in loop
for img in tqdm(img_list):
    img_path = os.path.join(img_dir, img)
    results = model.predict(img_path, imgsz=640,  save_txt=True, save_conf=False, show_labels=False, iou=0.6,conf=0.25)
    if results[0].masks is not None:
        boxes = results[0].boxes.xyxy  # Bounding box coordinates
        masks = results[0].masks.data.cpu()  # Segmentation masksave=True,s (moved to CPU and converted to NumPy array)
        image = cv2.imread(img_path)  # Read the original image
        im_array = results[0].plot(boxes= False, labels=False, probs=False)  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        #im.show()  # show image
        im.save(os.path.join(pred_img_save_dir, img+'.jpg'))
        # Plot the predicted mask
        predicted_mask = np.zeros_like(image[:, :, 0])
        for mask in masks:
            predicted_mask[np.where(mask)] = 255
            pd = Image.fromarray(predicted_mask[..., ::])  # RGB PIL image
            pd.save(os.path.join(pred_mask_save_dir , img+'.jpg')) # save image

### Now, the obtained predict folder from the above code which is in segments should be manually copied to "post_processing_direc" folder

In [ ]:
## For example

import shutil

predict_ws = '/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/Rural_WS/yolo8_segmentation_ws/runs/segment/predict'
new_directory = '/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/Rural_WS/yolo8_segmentation_ws/post_processing/Rural-M_2_ALUT_Test/'

shutil.move(predict_ws, new_directory)


### Reference Code Below

In [ ]:
import os
os.chdir('/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/All_Land_Use_Zones_WS/yolo8_segmentation_ws')

In [ ]:
home_direc = os.getcwd()
print(home_direc)

In [ ]:
from PIL import Image
from ultralytics import YOLO
from tqdm import tqdm
import cv2
import numpy as np

custom_model_direc = os.path.join(home_direc, 'runs/segment/train/weights/best.pt')
# Load a model
model = YOLO(custom_model_direc) 

#img_dir = os.path.join(home_direc, 'sample')
img_dir = os.path.join(home_direc, 'test/images')
img_list = os.listdir(img_dir)

pred_img_save_dir = os.path.join(home_direc, 'test_results_ws/alut-M_2_alut', 'predict_img') # image save folder
os.makedirs(pred_img_save_dir, exist_ok=True)

pred_mask_save_dir = os.path.join(home_direc, 'test_results_ws/alut-M_2_alut', 'predict_mask') # mask save folder
os.makedirs(pred_mask_save_dir, exist_ok=True)

# Predict for a single image in loop
for img in tqdm(img_list):
    img_path = os.path.join(img_dir, img)
    results = model.predict(img_path, imgsz=640, save_conf=True, show_labels=False, iou=0.5,conf=0.288)
    if results[0].masks is not None:
        boxes = results[0].boxes.xyxy  # Bounding box coordinates
        masks = results[0].masks.data.cpu()  # Segmentation masks (moved to CPU and converted to NumPy array)
        image = cv2.imread(img_path)  # Read the original image
        im_array = results[0].plot(boxes= False, labels=False, probs=False)  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        #im.show()  # show image
        im.save(os.path.join(pred_img_save_dir, img+'.jpg'))
        # Plot the predicted mask
        predicted_mask = np.zeros_like(image[:, :, 0])
        for mask in masks:
            predicted_mask[np.where(mask)] = 255
            pd = Image.fromarray(predicted_mask[..., ::])  # RGB PIL image
            pd.save(os.path.join(pred_mask_save_dir , img+'.jpg')) # save image